In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use(
    "https://raw.githubusercontent.com/erinhay/jolteon/refs/heads/main/notebooks/stylesheet/style.mplstyle"
)

In [2]:
# !ls /global/cfs/cdirs/desc-td/ELASTICC2/ELASTICC2_FINAL_Cepheid/

In [3]:
# !ls /global/cfs/cdirs/desc-td/ELASTICC2_parquet

In [4]:
# !ls /global/cfs/cdirs/desc-td/ELASTICC2_TRAIN02_parquet

In [5]:
letter_to_num = {"u": 0, "g": 1, "r": 2, "i": 3, "z": 4, "y": 5}
num_to_letter = {val: key for key, val in letter_to_num.items()}

# Peak wavelength for passband in Armstrong
lsst_wavelengths = {
    "u": 3670.69,
    "g": 4826.85,
    "r": 6223.24,
    "i": 7545.98,
    "z": 8590.90,
    "y": 9710.28,
}

In [6]:
import glob

In [7]:
glob.glob("/global/cfs/cdirs/desc-td/ELASTICC2_parquet/*.parquet")

['/global/cfs/cdirs/desc-td/ELASTICC2_parquet/PISN-STELLA_HYDROGENIC.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/PISN-MOSFIT.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/uLens-Single_PyLIMA.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/TDE.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/SNIcBL+HostXT_V19.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/KN_B19.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/uLens-Binary.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/SL-SNII.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/SNIc-Templates.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/SLSN-I+host.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/SNIa-SALT3.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/d-Sct.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/SNIb-Templates.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC2_parquet/SNII+HostXT_V19.parquet',
 '/global/cfs/cdirs/desc-td/ELASTICC

In [8]:
df = pd.read_parquet("/global/cfs/cdirs/desc-td/ELASTICC2_parquet/Cepheid.parquet")
# dftrain = pd.read_parquet("/global/cfs/cdirs/desc-td/ELASTICC2_TRAIN02_parquet/Cepheid.parquet")

In [9]:
from tqdm.auto import tqdm

In [10]:
# for path in tqdm(glob.glob("/global/cfs/cdirs/desc-td/ELASTICC2_parquet/*.parquet")):
#     df = pd.read_parquet(path)
#     if 53125011 in df["SNID"]:
#         print("SUCCESS", path)
#         break

In [11]:
for snid in df["SNID"]:
    chosen_df = df[df["SNID"] == snid].iloc[0]
    break

In [12]:
cols = ["MJD", "FLUXCAL", "FLUXCALERR", "BAND", "PHOTFLAG", "ZEROPT"]
dat = []
for col in cols:
    dat.append(chosen_df.loc[col])
dat = np.array(dat).T
lc = pd.DataFrame(data=dat, columns=cols).sort_values(by="MJD")

In [13]:
# only keep detections i.e. >5 SNR
lc = (
    lc[lc["PHOTFLAG"] == 4096]
    .reset_index(drop=True)
    .drop(["PHOTFLAG", "ZEROPT"], axis=1)
)

In [14]:
lc

,MJD,FLUXCAL,FLUXCALERR,BAND
0,60804.3538,-25319.236328,184.068268,Y
1,60804.3543,-25362.929688,179.296494,Y
2,60804.377,-26486.181641,183.822189,Y
3,60804.3774,-26504.472656,183.894058,Y
4,60813.385,-40566.726562,233.469772,Y
...,...,...,...,...
260,61879.2709,-1936.375977,23.415865,r
261,61879.4186,-17970.359375,107.335777,z
262,61885.2894,-4222.575195,28.622515,r
263,61889.2473,113.128914,5.441917,g


---

In [15]:
# !git clone https://github.com/alercebroker/pipeline.git

In [16]:
import sys
import os

# Path to where conda's pip installed the packages
packages_path = os.path.expanduser("~/packages")

# Add this path to the beginning of Python's search list
if packages_path not in sys.path:
    sys.path.insert(0, packages_path)

print(f"Added {packages_path} to sys.path")
# Optional: Print sys.path to verify
# print(sys.path)

Added /global/homes/c/chaini/packages to sys.path


In [17]:
from lc_classifier.examples.data import get_elasticc_example
from lc_classifier.features.composites.elasticc import ElasticcFeatureExtractor
from lc_classifier.features.core.base import AstroObject

In [18]:
feature_extractor = ElasticcFeatureExtractor()

In [19]:
def alerce_lcobj(detections, metadata):
    # assert detections["SNID"].unique().shape[0] == 1
    detections.rename(
        columns={
            "MJD": "mjd",
            "BAND": "fid",
            "FLUXCAL": "brightness",
            "FLUXCALERR": "e_brightness",
        },
        inplace=True,
    )
    detections["candid"] = None
    detections["tid"] = "elasticc_telescope"
    detections["sid"] = "elasticc_survey"
    detections["pid"] = "elasticc_program"

    metadata = metadata.loc[:, ["name", "value"]]

    detections["ra"] = float(metadata[metadata["name"] == "RA"]["value"].values[0])
    detections["dec"] = float(metadata[metadata["name"] == "DEC"]["value"].values[0])
    detections["unit"] = "diff_flux"

    is_detected = detections["PHOTFLAG"] > 0
    detections.drop(columns=["PHOTFLAG"], inplace=True)
    forced_photometry = detections[~is_detected]
    detections = detections[is_detected]

    metadata.loc[len(metadata)] = ["aid", "aid_example"]

    astro_object = AstroObject(
        detections=detections, forced_photometry=forced_photometry, metadata=metadata
    )

    return astro_object

In [20]:
def alerce_feature_listobjs(
    astro_objects, feature_extractor=ElasticcFeatureExtractor(), progress_bar=False
):
    # astro_objects is list of astro_object e.g. [astro_object1, astro_object2, etc.]
    feature_extractor.compute_features_batch(
        [elasticc_object], progress_bar=progress_bar
    )
    return astro_objects

In [44]:
def get_detections(chosen_df, snid):
    detections_colnames = ["SNID", "MJD", "BAND", "FLUXCAL", "FLUXCALERR", "PHOTFLAG"]
    tot_pts = chosen_df["MJD"].shape[0]
    dat = []
    dat.append([snid] * (tot_pts))
    for col in detections_colnames[1:]:
        dat.append(chosen_df.loc[col])
    dat = np.array(dat).T
    detections = (
        pd.DataFrame(data=dat, columns=detections_colnames)
        .sort_values(by="MJD")
        .reset_index(drop=True)
    )
    return detections


def get_meta(chosen_df, snid):
    metacols = [
        "SNID",
        "RA",
        "DEC",
        "SNTYPE",
        "NOBS",
        "PTROBS_MIN",
        "PTROBS_MAX",
        "MWEBV",
        "MWEBV_ERR",
        "REDSHIFT_HELIO",
        "REDSHIFT_HELIO_ERR",
        "REDSHIFT_FINAL",
        "REDSHIFT_FINAL_ERR",
        "VPEC",
        "VPEC_ERR",
    ]

    names = chosen_df.loc[metacols].index.to_numpy()
    values = chosen_df.loc[metacols].to_numpy()
    dat = np.stack([names, values]).T
    metadata = pd.DataFrame(data=dat, columns=["name", "value"])
    return metadata


def get_sniddata(df_parquet, snid):
    chosen_df = df_parquet[df_parquet["SNID"] == snid].iloc[0]
    detections = get_detections(chosen_df, snid)
    metadata = get_meta(chosen_df, snid)

    return detections, metadata

In [45]:
# get_detections(df, snid_val)

In [46]:
detections = pd.read_csv(
    "https://github.com/alercebroker/pipeline/raw/refs/heads/main/lc_classifier/lc_classifier/examples/elasticc/elasticc_snid_53125011_detections.csv"
)
metadata = pd.read_csv(
    "https://github.com/alercebroker/pipeline/raw/refs/heads/main/lc_classifier/lc_classifier/examples/elasticc/elasticc_snid_53125011_metadata.csv"
)

In [47]:
elasticc_object = alerce_lcobj(detections, metadata)

In [41]:
if elasticc_object.redshift_name is None:
    pass

AttributeError: 'AstroObject' object has no attribute 'redshift_name'

In [35]:
# # PeriodExtractor: Only used for logging
# elasticc_object.metadata[elasticc_object.metadata["name"] == "aid"]["value"].values[0]

'aid_example'

In [32]:
elasticc_object.metadata

,name,value
0,SNID,53125011
1,IAUC,NaN
2,FAKE,2
3,RA,82.50327958115051
4,DEC,-8.543473129539061
...,...,...
166,SIM_GALFRAC_i,22.15407943725586
167,SIM_GALFRAC_z,29.115129470825195
168,SIM_GALFRAC_Y,26.309558868408203
169,SIM_SUBSAMPLE_INDEX,-9


In [23]:
feature_extractor = ElasticcFeatureExtractor()
feature_extractor.compute_features_batch([elasticc_object], progress_bar=True)
print(elasticc_object.features)

100%|██████████| 1/1 [00:06<00:00,  6.27s/it]

                            name      value          fid              sid  \
0                            u-g        NaN  u,g,r,i,z,Y  elasticc_survey   
1                            g-r        NaN  u,g,r,i,z,Y  elasticc_survey   
2                            r-i   0.818937  u,g,r,i,z,Y  elasticc_survey   
3                            i-z   1.383519  u,g,r,i,z,Y  elasticc_survey   
4                            z-Y        NaN  u,g,r,i,z,Y  elasticc_survey   
..                           ...        ...          ...              ...   
59  median_brightness_after_band  11.560403            Y  elasticc_survey   
0                       Timespan  40.979100         None  elasticc_survey   
0                   Coordinate_x   0.129022         None  elasticc_survey   
1                   Coordinate_y   0.980451         None  elasticc_survey   
2                   Coordinate_z  -0.148560         None  elasticc_survey   

   version  
0    1.0.0  
1    1.0.0  
2    1.0.0  
3    1.0.0  
4    1.0.0